In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.vision.all import *

#For Image Augmentation
import albumentations

#Std import for numbers and dataframe
import numpy as np # linear algebra
import pandas as pd

In [ ]:
class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx,order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

In [ ]:
def get_train_aug(): return albumentations.Compose([
            albumentations.RandomResizedCrop(256,256),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5)
])

def get_valid_aug(): return albumentations.Compose([
    albumentations.CenterCrop(256,256, p=1.),
    albumentations.Resize(256,256)
], p=1.)

item_tfms = AlbumentationsTransform(get_train_aug(), get_valid_aug())


In [ ]:
def get_x(df): return TRAIN_DIR/df['image_id']
def get_y(df): return df['label']
def splitter(df):
    train = df.index[~df['is_val']].tolist()
    valid = df.index[df['is_val']].tolist()
    return train,valid

In [ ]:
# load the pickle file you exported previously
path = Path('/kaggle/input/casava-modelpkl')
path.ls(file_exts='.pkl')

learn_inf = load_learner(path/'export_best_model.pkl')

In [ ]:
path = Path('/kaggle/input/cassava-leaf-disease-classification')
DATA_DIR = path
TRAIN_DIR = DATA_DIR/'train_images'
TEST_DIR = DATA_DIR/'test_images'
TRAIN_DIR,TEST_DIR

In [ ]:
sample_df = pd.read_csv(DATA_DIR/'sample_submission.csv')

In [ ]:
sample_df.head(10)

In [ ]:
sample_copy = sample_df.copy()
sample_copy['image_id'] = sample_copy['image_id'].apply(lambda x: f'../test_images/{x}')

In [ ]:
test_dl = learn_inf.dls.test_dl(sample_copy)

In [ ]:
preds, _ = learn_inf.tta(dl=test_dl, n=15, beta=0)

In [ ]:
preds

In [ ]:
sample_df['label'] = preds.argmax(dim=-1).numpy()

In [ ]:
sample_df.to_csv('submission.csv',index=False)